### Code that asks Chat GPT to assess if the Reddit comment is about crime and then analyze the sentiment on crime

In [1]:
# load api key
# load library to get environmental files
import os
from dotenv import load_dotenv


# load keys from  environmental var
load_dotenv() # .env file in cwd
gpt_key = os.environ.get("gpt_key") 

In [3]:
import requests 
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
## function to query a response from Chat GPT
def ask_chat(question, api_key):
    # define headers
    headers = {
        "Authorization": f"Bearer {gpt_key}",
        "Content-Type": "application/json",
    }

    info = {
     "model": "gpt-4o-mini",
     "messages": [{"role": "user", "content": question}],
     "temperature": 0
    }
    ##Get the response
    response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, json = info)
    ##Convert to json
    response_json = response.json()
    return response_json['choices'][0]['message']['content'].strip()

In [21]:
##open coded csv file
test_data = pd.read_csv("./data/reddit/comments_parsed.csv", skiprows=5250, nrows=150, names=["x","y","comment_id","submission_id","author","body","score","year","month","subreddit","type"])
test_data.head()

,x,y,comment_id,submission_id,author,body,score,year,month,subreddit,type
0,5257,5367,kjsmj2q,1abwq88,westgazer,Did you know crime is also real bad in republi...,0,2024,1,1,1
1,5258,5368,kju5k0t,1abwq88,burgermac12,You should just go fuck off,3,2024,1,1,1
2,5259,5369,kjtaik8,1abwq88,Sunbeamsoffglass,DC has had more murders in 2023 than any year ...,2,2024,1,1,1
3,5260,5370,kjqmxlz,1abwq88,burgermac12,You’re an idiot,25,2024,1,1,1
4,5261,5371,kjssiqh,1abwq88,Reeetankiesbtfo,"Arlington had zero murders in 2020 and 2021, a...",3,2024,1,1,1


In [23]:
test_data.shape

(150, 11)

In [25]:
import time
##loop through the coded data to test gpt's accuracy with prompt
answer = []
for j in tqdm(range(len(test_data))):
    try:
        base_question = "Is this text about neighborhood safety in DC? \
        Answer only with a number: 1 if about safety in DC, 0 if not. \
        If the answer is 1, how safe do you think the author feels? \
        Answer with only a number on a scale between 1 (very unsafe) and 10 (very safe) \
            Here is the text: "
        text = test_data.loc[j, "body"]
        full_question = base_question + str(text)
        answer.append(ask_chat(full_question, gpt_key))
        time.sleep(1)
    except:
        answer.append(np.nan)

100%|█████████████████████████████████████████| 150/150 [01:09<00:00,  2.15it/s]


In [27]:
answer[149]

nan

In [15]:
len(answer)

50

In [17]:
test_data['gpt_score'] = answer

In [137]:
test_data.head(25)

,x,y,comment_id,submission_id,author,body,score,year,month,subreddit,type,gpt_score
0,5057,5156,l6ciaxn,1d474to,BlkNtvTerraFFVI,I'm fine with people talking about crime but p...,4,2024,5,1,1,1 \n5
1,5058,5157,l6csge0,1d474to,PanAmargo,DC was like 5th in the country for murders las...,3,2024,5,1,1,1 \n3
2,5059,5158,l6duoac,1d474to,IcyWillow1193,the worst of them pretty obviously live nowher...,5,2024,5,1,1,0
3,5060,5159,l6e3qxd,1d474to,douggie84,You made a throw-away account for this because...,2,2024,5,1,1,0
4,5061,5160,l6cdb6l,1d474to,little_bird_vagabond,"I live in nova, but I worked in DC until recen...",2,2024,5,1,1,1 \n7
5,5062,5161,l6hmqi1,1d474to,PooEating007,"Yeah, fuck that, I’m not going anywhere. If yo...",1,2024,5,1,1,1 \n3
6,5063,5163,l6d6cl9,1d474to,Substantial_Bet_3351,Tf is wrong with you. It is a warzone dummy. P...,0,2024,5,1,1,1 \n3
7,5064,5164,l6dl01n,1d474to,NaN,I live in Maryland.\n\nBut denying the crappy ...,0,2024,5,1,1,0
8,5065,5165,l6c8m8a,1d474to,eternalkushcloud,"they have a victim and crime fetish, it's kind...",-5,2024,5,1,1,0
9,5066,5166,l6crvlq,1d474to,4look4rd,"DC is the best area in the US, it still sucks ...",0,2024,5,1,1,0


In [19]:
test_data.to_csv("./data/reddit/gpt_output_4.csv")